# Re-estimation of Analysis

In [ ]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import statsmodels

In [ ]:
#%run Documents/GitHub/Thesis/DALY_WASH.ipynb
#%run Documents/GitHub/Thesis/ucdp_thesis.ipynb
#%run Documents/GitHub/Thesis/controls.ipynb
#%run Documents/GitHub/Thesis/merge_thesis.ipynb

In [ ]:
##set a working directory
user = os.path.expanduser('~')
display(user)
os.makedirs(f'{user}/Desktop/private/thesis/', exist_ok=True)
path = os.chdir(f'{user}/Desktop/'+'private/thesis/')


In [ ]:
data =pd.read_csv('data_final.csv')
print(data.columns)
display(data.describe())

In [ ]:
#import population size
dta = pd.read_csv('pop_size.csv',skiprows= 4)
display(dta.columns)
display(dta)

#clean up
dta = dta.drop(columns=['Country Name','Indicator Name', 'Indicator Code','Unnamed: 67'])
dta = dta.rename(columns= {'Country Code':'iso'})
dta = dta.melt(id_vars='iso',var_name='year_id',value_name='pop_size')
dta['year_id'] = dta['year_id'].astype(str).astype(int)

## only years from 1990 onwards
dta = dta.loc[dta['year_id'] >= 1990]
dta = dta.loc[dta['year_id'] <= 2019]
#only countries in SSA
dta = dta.loc[dta['iso'].isin(['AGO', 'BDI', 'BEN', 'BFA', 'BWA', 'CAF', 'CIV', 'CMR', 'COD',
       'COG', 'COM', 'CPV', 'DJI', 'ERI', 'ETH', 'GAB', 'GHA', 'GIN',
       'GMB', 'GNB', 'GNQ', 'KEN', 'LBR', 'LSO', 'MDG', 'MLI', 'MOZ',
       'MRT', 'MWI', 'NAM', 'NER', 'NGA', 'RWA', 'SDN', 'SEN', 'SLE',
       'SOM', 'SSD', 'STP', 'SWZ', 'TCD', 'TGO', 'TZA', 'UGA', 'ZAF',
       'ZMB', 'ZWE'])]

#save
dta.to_csv('data_pop_size.csv')

In [ ]:
#merge the datasets on iso and year
data = pd.merge(data, dta, how="outer", on=["iso", "year_id"])
print(data.year_id.unique())

# Vizualisation

In [ ]:
#time trend of the data
grouped = data.groupby(['iso', 'year_id']).agg({'daly_all': 'sum'})
top_countries = grouped.groupby('iso').agg({'daly_all': 'sum'}).nlargest(10, 'daly_all').index.tolist()
filtered = grouped[grouped.index.get_level_values('iso').isin(top_countries)]

pivoted = filtered.reset_index().pivot(index='year_id', columns='iso', values='daly_all')
pivoted.plot.area(stacked=False)

plt.xlabel('Year')
plt.ylabel('DALY/ 100.000 people')
plt.savefig('timetrend_daly.png')
# Show the plot
plt.show()

In [ ]:
#time trend of the data
grouped = data.groupby(['iso', 'year_id']).agg({'best_log': 'sum'})
top_countries = grouped.groupby('iso').agg({'best_log': 'sum'}).nlargest(10, 'best_log').index.tolist()
filtered = grouped[grouped.index.get_level_values('iso').isin(top_countries)]

# Pivot the data to create a column for each donor and a row for each year
pivoted = filtered.reset_index().pivot(index='year_id', columns='iso', values='best_log')
pivoted.plot.area(stacked=False)

plt.xlabel('Year')
plt.ylabel('Conflict fatalities (best estimate)')
plt.savefig('timetrend_conflict.png')

# Show the plot
plt.show()

In [ ]:
# combined plot
grouped_1 = data.groupby(['year_id']).agg({'best_log': 'sum'})
grouped_2 = data.groupby(['year_id']).agg({'daly_all': 'sum'})
grouped_1 = grouped_1.reset_index()
grouped_2 = grouped_2.reset_index()
display(grouped_1)
display(grouped_2)
# create figure and axis objects with subplots()
fig,ax = plt.subplots()
# make a plot
ax.plot(grouped_2.year_id,
        grouped_2.daly_all,
        color="green")
# set x-axis label
ax.set_xlabel("year", fontsize = 10)
# set y-axis label
ax.set_ylabel("DALY/ 100.000 people",
              color="green",
              fontsize=10)


# twin object for two different y-axis on the sample plot
ax2=ax.twinx()
# make a plot with different y-axis using second axis object
ax2.plot(grouped_1.year_id, grouped_1["best_log"],color="blue")
ax2.set_ylabel("conflict fatalities",color="blue",fontsize=10)
plt.savefig('timetrend_combined.png')
plt.show()

# start the reestimation by defining type of conflict

In [ ]:
#define datasets with the different types of conflict
data_sb = data.loc[data['type_of_violence'] == 'sb']
display(data_sb)

data_ns = data.loc[data['type_of_violence'] == 'ns']
display(data_ns)

data_os = data.loc[data['type_of_violence'] == 'os']
display(data_os)

# Model with fatalities estimate

In [ ]:
#the country fixed effects model LAGS
from linearmodels.panel import PanelOLS
data1 = data.set_index(['iso','year_id'])
y= data1['daly_all']
x= data1[['best_log','lag_best_1','lag_best_2','lag_best_3','lag_best_4','lag_best_5','lag_best_6','lag_best_7','lag_best_8','lag_best_9','lag_best_10']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
fat = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_conflict_history.txt', 'w') as f:
    f.write(fat.summary.as_text())
print(fat)

In [ ]:
#the country fixed effects model LAGS
from linearmodels.panel import PanelOLS
data1 = data_sb.set_index(['iso','year_id'])
y= data1['daly_all']
x= data1[['best_log','lag_best_1','lag_best_2','lag_best_3','lag_best_4','lag_best_5','lag_best_6','lag_best_7','lag_best_8','lag_best_9','lag_best_10']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
fat_sb = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_conflict_history_sb.txt', 'w') as f:
    f.write(fat_sb.summary.as_text())
print(fat_sb)

In [ ]:
#the country fixed effects model LAGS
from linearmodels.panel import PanelOLS
data1 = data_os.set_index(['iso','year_id'])
y= data1['daly_all']
x= data1[['best_log','lag_best_1','lag_best_2','lag_best_3','lag_best_4','lag_best_5','lag_best_6','lag_best_7','lag_best_8','lag_best_9','lag_best_10']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
fat_os = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_conflict_history_os.txt', 'w') as f:
    f.write(fat_os.summary.as_text())
print(fat_os)

In [ ]:
#the country fixed effects model LAGS
from linearmodels.panel import PanelOLS
data1 = data_ns.set_index(['iso','year_id'])
y= data1['daly_all']
x= data1[['best_log','lag_best_1','lag_best_2','lag_best_3','lag_best_4','lag_best_5','lag_best_6','lag_best_7','lag_best_8','lag_best_9','lag_best_10']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
fat_ns = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_conflict_history_ns.txt', 'w') as f:
    f.write(fat_os.summary.as_text())
print(fat_ns)

# specification models

# STANDARD

In [ ]:
# specification models without conflict history
#the country fixed effects model STANDARD
from linearmodels.panel import PanelOLS
data1 = data.set_index(['iso','year_id'])
y= data1['daly_all']
x= data1[['best_log','pop_size','gdp_log', 'age0014_value','xpd_gdp_value','lag_best_1','lag_best_2','lag_best_3','lag_best_4','lag_best_5','lag_best_6','lag_best_7','lag_best_8','lag_best_9','lag_best_10']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
standard_spec = mod.fit(cov_type='clustered', cluster_entity=True)
with open('resuts_standard.txt', 'w') as f:
    f.write(standard_spec.summary.as_text())
print(standard_spec)

In [ ]:
# specification models without conflict history
#the country fixed effects model STANDARD
from linearmodels.panel import PanelOLS
data1 = data.set_index(['iso','year_id'])
y= data1['daly_all']
x= data1[['best_log','pop_size','gdp_log','lag_best_1','lag_best_2','lag_best_3','lag_best_4','lag_best_5','lag_best_6','lag_best_7','lag_best_8','lag_best_9','lag_best_10']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
standard_spec = mod.fit(cov_type='clustered', cluster_entity=True)
with open('resuts_standard.txt', 'w') as f:
    f.write(standard_spec.summary.as_text())
print(standard_spec)

In [ ]:
data.columns

# HEALTH

In [ ]:
#logtransform the healthworker variable
data['healthworker_all_log'] = np.log(data['ihme_healthworkers_all_mean'])
display(data)

In [ ]:
#the country fixed effects model HEALTH
from linearmodels.panel import PanelOLS
data1 = data.set_index(['iso','year_id'])
y= data1['daly_all']
x= data1[['best_log','lag_best_1','lag_best_2','lag_best_3','lag_best_4','lag_best_5','lag_best_6','lag_best_7','lag_best_8','lag_best_9','lag_best_10','dah_total_log','healthworker_all_log']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
health_spec = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_health_specification.txt', 'w') as f:
    f.write(health_spec.summary.as_text())
print(health_spec)

## DISASTER

In [ ]:
#logtransform the death variable
data['death_tot_log'] = np.log1p(data['death_tot'])

In [ ]:
#the country fixed effects model DISASTER
from linearmodels.panel import PanelOLS
data1 = data.set_index(['iso','year_id'])
y= data1['daly_all']
x= data1[['best_log','lag_best_1','lag_best_2','lag_best_3','lag_best_4','lag_best_5','lag_best_6','lag_best_7','lag_best_8','lag_best_9','lag_best_10','access_drinkwater_index', 'access_sani_index', 'death_tot_log','gdis_count_lag']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
disaster_spec = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_disaster_specification.txt', 'w') as f:
    f.write(disaster_spec.summary.as_text())
print(disaster_spec)

## all

In [ ]:
#the country fixed effects model ALL
from linearmodels.panel import PanelOLS
data2 = data.set_index(['iso','year_id'])
y= data2['daly_all']
x= data2[['best_log','lag_best_1','lag_best_2','lag_best_3','lag_best_4',
          'lag_best_5','lag_best_6','lag_best_7','lag_best_8','lag_best_9',
          'lag_best_10','pop_size','gdp_log', 'age0014_value','xpd_gdp_value',
          'dah_total_log','healthworker_all_log','access_drinkwater_index', 
          'access_sani_index', 'death_tot_log','gdis_count_lag']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
model_overfit = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_all.txt', 'w') as f:
    f.write(model_overfit.summary.as_text())
print(model_overfit)

## now do it for the lags separately

normal fat estimate

In [ ]:
#the country fixed effects model STANDARD
from linearmodels.panel import PanelOLS
data1 = data.set_index(['iso','year_id'])
y= data1['daly_all']
x= data1[['best_log','gdp_log', 'age0014_value','xpd_gdp_value',]]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
standard_spec = mod.fit(cov_type='clustered', cluster_entity=True)
with open('resuts_standard.txt', 'w') as f:
    f.write(standard_spec.summary.as_text())
print(standard_spec)

In [ ]:
#the country fixed effects model HEALTH
from linearmodels.panel import PanelOLS
data1 = data.set_index(['iso','year_id'])
y= data1['daly_all']
x= data1[['best_log','dah_total_log','healthworker_all_log']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
health_spec = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_health_specification.txt', 'w') as f:
    f.write(health_spec.summary.as_text())
print(health_spec)

In [ ]:
#the country fixed effects model DISASTER
from linearmodels.panel import PanelOLS
data1 = data.set_index(['iso','year_id'])
y= data1['daly_all']
x= data1[['best_log','access_drinkwater_index', 'access_sani_index', 'death_tot_log','gdis_count_lag']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
disaster_spec = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_disaster_specification.txt', 'w') as f:
    f.write(disaster_spec.summary.as_text())
print(disaster_spec)

lag 5

In [ ]:
# lag 5
#the country fixed effects model standard
from linearmodels.panel import PanelOLS
data2 = data.set_index(['iso','year_id'])
y= data2['daly_all']
x= data2[['lag_best_5','gdp_log', 'age0014_value','xpd_gdp_value']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
standard_lag5 = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_all.txt', 'w') as f:
    f.write(standard_lag5.summary.as_text())
print(standard_lag5)

In [ ]:
# lag 5
#the country fixed effects model health
from linearmodels.panel import PanelOLS
data2 = data.set_index(['iso','year_id'])
y= data2['daly_all']
x= data2[['lag_best_5','dah_total_log','healthworker_all_log']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
health_lag5 = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_all.txt', 'w') as f:
    f.write(health_lag5.summary.as_text())
print(health_lag5)

In [ ]:
# lag 5
#the country fixed effects model disaster
from linearmodels.panel import PanelOLS
data2 = data.set_index(['iso','year_id'])
y= data2['daly_all']
x= data2[['lag_best_5','access_drinkwater_index', 'access_sani_index', 'death_tot_log','gdis_count_lag']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
disaster_lag5 = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_all.txt', 'w') as f:
    f.write(disaster_lag5.summary.as_text())
print(disaster_lag5)

lag 10

In [ ]:
# lag 10
#the country fixed effects model standard
from linearmodels.panel import PanelOLS
data2 = data.set_index(['iso','year_id'])
y= data2['daly_all']
x= data2[['lag_best_10','gdp_log','age0014_value','xpd_gdp_value']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
standard_lag10 = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_all.txt', 'w') as f:
    f.write(standard_lag10.summary.as_text())
print(standard_lag10)

In [ ]:
# lag 10
#the country fixed effects model health
from linearmodels.panel import PanelOLS
data2 = data.set_index(['iso','year_id'])
y= data2['daly_all']
x= data2[['lag_best_10','dah_total_log','healthworker_all_log']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
health_lag10 = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_all.txt', 'w') as f:
    f.write(health_lag10.summary.as_text())
print(health_lag10)

In [ ]:
#the country fixed effects model disaster
from linearmodels.panel import PanelOLS
data2 = data.set_index(['iso','year_id'])
y= data2['daly_all']
x= data2[['lag_best_10','access_drinkwater_index', 'access_sani_index', 'death_tot_log','gdis_count_lag']]
mod = PanelOLS(y,x, entity_effects=True,time_effects=False)
disaster_lag10 = mod.fit(cov_type='clustered', cluster_entity=True)
with open('results_all.txt', 'w') as f:
    f.write(disaster_lag10.summary.as_text())
print(disaster_lag10)

# plot the estimates for the lags to get the temporal effect

In [ ]:
#choose the version of dependent, this will run only one type of formula at a time, so no groupings by age and sex
#additionaly this was made for civil war binary variable, but just in case if we want to do ged, the variable_portion_to_strip should enable the use of something where variable names are identical and t0 or tl1 carry the information on temporal placement of the variable. 
version_of_dependent = ''
variable_portion_to_strip = 'civil_war_binary_'
#blank dataframe

for ghe_number in ['10', '600', '1510']:
    for sex in ['btsx']:
        for age_group in ['all_ages']:
            var = f"{version_of_dependent}level1_ghe{ghe_number}_{age_group}_{sex}_rate"
            indep=violence_variables_ln_special+violence_splag+other_controls
            indep_in_formula = '+'.join(indep)
            
            
            
            model2 = ols(formula=var+f"~ {indep_in_formula} +C(year_id)+C(country_id)",data=test).fit()
            fe_groups = test.copy()
            
            for i in [var,'country_id', 'year_id']+indep:
                fe_groups = fe_groups[pd.notnull(fe_groups[i])]
                
            model3 = ols(formula=var+f"~ {indep_in_formula} + C(year_id)+C(country_id)",data=fe_groups).fit(cov_type='cluster', cov_kwds={'groups': fe_groups ['country_id']})
            
            se = round(model3.bse,3)
            pvals = round(model3.pvalues,3)
            coeff = round(model3.params,10)
            nobs = int(model3.nobs)
            r2 = round(model3.rsquared_adj, 2)
            dep = var
            
            #things I want to store, age, sex, and ghe type
            results_df = pd.DataFrame({"coeff":coeff,"s.e.":se, "pvals":pvals})
            results_df = results_df.drop('pvals', axis=1)
        

            results = results_df.query(f"index in {violence_variables_ln_special}")
            results = results.copy()
            results['order'] = results.index.str.strip(variable_portion_to_strip)
            
            results['ll'] = results['coeff'] - 1.96*(results['s.e.'])
            results['hl'] = results['coeff'] + 1.96*(results['s.e.'])

            data = results.copy()
            #the following creates the error term which will be used to make the confidence interval
            errors = data['coeff'] - data['ll']
            data['errors'] = errors

            #the following should order things together
            #for our case we want them to be age groups
            #we need to specify the order of age groups, i want all ages to occur last

            custom_dict = dict(zip([f"tl{i}" for i in range(10, 0, -1)]+[f"t{j}" for j in range(0,11)], range(-10, 11)))
            data = data.sort_values(by=['order'], key = lambda x: x.map(custom_dict))
            data['variables'] = data.index
            
            sns.set_context('poster')

            #figure, axes, and plot
            fig,ax = plt.subplots(figsize = (15,10))
            
            
            data.plot(x='variables', y='coeff', kind='bar', ax=ax, color=['orange']*10 + ['steelblue']*(len(data)-10), fontsize=22, ecolor='black',capsize=0,yerr='errors', legend=False)
            
            # Set title & labels
            plt.title('Coefficients of Features w/ 95% Confidence Intervals',fontsize=30)
            ax.set_ylabel('Coefficients',fontsize=22)
            ax.set_xlabel('Time',fontsize=22)
    
            # Coefficients
            ax.scatter(x=pd.np.arange(data.shape[0]), marker='o', s=80, y=data['coeff'], color='black')
            
            ax.set_xticklabels([custom_dict.get(label) for label in data['order']])

    
            # Line to define zero on the y-axis
            ax.axhline(y=0, linestyle='--', color='red', linewidth=1)
            
            filename = f"dep_{version_of_dependent}level1_ghe{ghe_number}_{age_group}_{sex}_indep_{variable_portion_to_strip}tlag_tlead_coefficient_plot.png"
            
            plt.savefig(f"{output}figures/{figure_folder}/{filename}", dpi = 300)
            plt.close()